In [1]:
import os
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import datetime
import matplotlib.pyplot as plt

In [2]:
edw = pd.read_csv(edw_path)
o2 = pd.read_csv(o2_path)

In [3]:
o2['RECORDED_TIME'] = pd.to_datetime(o2['RECORDED_TIME'])
o2['calendar_day'] = [np.datetime64(rt, 'D') for rt in o2.RECORDED_TIME]
o2['patient_calendar_day'] = [str(r.pt_study_id) + str(r.calendar_day) for i,r in o2.iterrows()]
o2['time_of_day'] = [x.astype('datetime64[s]').astype(object).time() for x in o2.RECORDED_TIME.values]

## Same-day fails

In [4]:
def check_failed_extubation(sub):
    if len(sub) == 0:
        return False
    extub_yes = ((sub.DISP_NAME == "Extubation") & (sub.MEAS_VALUE == "Yes"))
    vent_dc = ((sub.DISP_NAME == 'Vent D/C') | (sub.DISP_NAME == 'Vent D/C Time'))
    extub_check =  np.where(extub_yes | vent_dc)[0]

    vent_display = ((sub.DISP_NAME == 'O2 Device') & (sub.MEAS_VALUE == 'Ventilator'))
    vent_yes = (sub.DISP_NAME == 'Ventilator')
    intub_yes = np.where(vent_display | vent_yes)[0]

    pt_day = sub.patient_day.values[0]
    stay = pt_day.rsplit('/',1)[0]
    day = pt_day.rsplit('/',1)[1]
    next_day = f"{stay}/{int(day)+1}"
    if next_day in list(edw.patient_day):
        next_day_status = edw[edw.patient_day == next_day].extubation_status.values[0]
    else:
        next_day_status = 'missing'


    # at least three markers of intubation after extubation
    if len(extub_check) > 0:
        if next_day_status == 'intubated':
            return True
        elif len(intub_yes) > 0: 
            num_greater = len([x for x in intub_yes if x > max(extub_check)])
            return num_greater >= 3
        else:
            return False
    else:
        return False

In [5]:
check_fail = []
intub_days = edw[edw.extubation_status == 'intubated'].patient_day.unique()
for day in intub_days:
    sub = o2[(o2.patient_day == day) & (o2['time_of_day'] >= datetime.time(8, 0))]
    if check_failed_extubation(sub):
        check_fail.append(day)

In [6]:
len(check_fail)

41

In [8]:
edw.loc[edw.patient_day.isin(check_fail),'extubation_status'] = 'same-day fail'

## Flag days that have the trach collar

In [9]:
def trach_check(sub,min_cts=3): 
    trach_indices = np.where((sub.DISP_NAME == 'O2 Device') & (sub.MEAS_VALUE == 'Trach collar'))
    if len(trach_indices) > 0:
        if len(trach_indices[0]) >= min_cts:
            return True
        else:
            return False
    else:
        return False

In [10]:
trach_pts = []
for pt in o2.pt_study_id.unique():
    sub = o2[o2.pt_study_id == pt]
    if trach_check(sub,min_cts=3):
        trach_pts.append(pt)

In [11]:
len(trach_pts)

126

In [12]:
trach_df = pd.read_excel(trach_path)
trach_df = trach_df.iloc[7:]
trach_df.columns = trach_df.iloc[0]
trach_df = trach_df.iloc[1:]
trach_df['trach_status'] = [True if 'Tracheostomy' in x else False for x in trach_df['PROPERTIES DISPLAY']]

In [13]:
trach_pull = trach_df['pt study id'].unique()
trach_pull = [int(x) for x in trach_pull]

In [14]:
edw.calendar_day = pd.to_datetime(edw.calendar_day)

In [15]:
additional_trach_pts = []
for pt in trach_pull:
    if pt in list(edw.pt_study_id.unique()) and pt not in trach_pts:
        placement_date = trach_df[trach_df['pt study id'] == str(pt)]['Placement Date'].values[0]
        if pd.isna(placement_date):
            continue
        last_date = edw[edw.pt_study_id == pt].calendar_day.max()
        first_date = edw[edw.pt_study_id == pt].calendar_day.min()
        if placement_date >= first_date and placement_date <= last_date:
            additional_trach_pts.append(pt)

In [16]:
len(additional_trach_pts)

84

In [17]:
all_flagged = trach_pts.copy()
all_flagged.extend(additional_trach_pts)

In [18]:
edw['trach_received'] = [True if x in all_flagged else False for x in edw.pt_study_id]

In [19]:
edw['Intubation_flag'] = [1 if x == 'intubated' else 0 for x in edw.extubation_status]

In [20]:
new_status_dict = {
    r.patient_day:r.extubation_status for i,r in edw.iterrows()
}
o2['extubation_status'] = o2['patient_day'].map(new_status_dict)

Save intermediate files

In [23]:
# o2.to_csv()

In [24]:
# edw.to_csv()